<a href="https://colab.research.google.com/github/safdarjung/projects/blob/main/Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !unzip -j /content/assignment.zip/
# ! pip install streamlit
# !pip install newspaper3k
# !pip install pyphen


In [ ]:
'''
importing required libraries
'''

import pandas as pd
import numpy as np
import nltk
import os
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import nltk
from nltk.tokenize import sent_tokenize
from newspaper import Article, ArticleException    # used for extracting information from articles
import pyphen

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# reading the url file

Input = pd.read_excel('/content/Input.xlsx')
Input.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [ ]:
Input.shape

(100, 2)

In [ ]:
urls = Input['URL'].tolist()       # urls list

In [ ]:
'''
extracting title and text from the urls in a list
also handling if the url data cannot be downloaded
applying the func to the urls list
'''

def extract_article_info(url):
    try:
        article = Article(url, language='en')
        article.download()
        article.parse()
        article.nlp()
        return article.title, article.text
    except ArticleException as e:
        print(f"Failed to download article from {url}: {e}")
        return None, None

titles_and_texts = []
for url in urls:
    title, text = extract_article_info(url)
    if title and text: # Check if both title and text are not None
        titles_and_texts.append((title, text))


Failed to download article from https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/: Article `download()` failed with 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/ on URL https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Failed to download article from https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/: Article `download()` failed with 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/ on URL https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/


In [ ]:
from newspaper import Article, ArticleException


def extract_article_info(url):
    try:
        article = Article(url, language='en')
        article.download()
        article.parse()
        article.nlp()
        return article.title, article.text
    except ArticleException as e:
        print(f"Failed to download article from {url}: {e}")
        return None, None

titles_and_texts = []
for url in urls:
    title, text = extract_article_info(url)
    if title and text: # Check if both title and text are not None
        titles_and_texts.append((title, text))


Failed to download article from https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/: Article `download()` failed with 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/ on URL https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Failed to download article from https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/: Article `download()` failed with 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/ on URL https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/


In [ ]:
url_df=pd.DataFrame(titles_and_texts)
url_df.shape

(98, 2)

In [ ]:
'''
loading the stopwords provided as .txt files
'''

def load_stopwords(file_paths):
    stopwords = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='latin1') as file:
            stopwords.extend(file.read().split())
    return stopwords

stopwords_file_paths = [
    '/content/StopWords_Auditor.txt',
    '/content/StopWords_Currencies.txt',
    '/content/StopWords_DatesandNumbers.txt',
    '/content/StopWords_Generic.txt',
    '/content/StopWords_GenericLong.txt',
    '/content/StopWords_Geographic.txt',
    '/content/StopWords_Names.txt'
]

stopwords = load_stopwords(stopwords_file_paths)


In [ ]:
'''
preprocessing the text data extracted from the articles
lowercase,
remove punctuations and numbers
tokenizing --> string to list
removing stopwords and lemmatizinng the remaining words
'''

lemma = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]+', ' ', text).strip()
    tokens = word_tokenize(text)
    tokens = [lemma.lemmatize(token) for token in tokens if token not in stopwords]
    return tokens


POSITIVE SCORE and NEGATIVE SCORE: These can be calculated using sentiment analysis techniques. For simplicity, let's use a basic approach based on word counts.
POLARITY SCORE and SUBJECTIVITY SCORE: These can be calculated using the TextBlob library.
AVG SENTENCE LENGTH: Calculate the average length of sentences.
PERCENTAGE OF COMPLEX WORDS: Calculate the percentage of words that are considered complex.
FOG INDEX: Calculate the Fog Index, which is a readability test.
AVG NUMBER OF WORDS PER SENTENCE: Calculate the average number of words per sentence.
COMPLEX WORD COUNT: Count the number of complex words.
WORD COUNT: Count the total number of words.
SYLLABLE PER WORD: Calculate the average number of syllables per word.
PERSONAL PRONOUNS: Count the number of personal pronouns.
AVG WORD LENGTH: Calculate the average length of words.

In [ ]:
'''
calculating positiva and negative scores of the preprocessed text
using the positive and negative words in the given .txt files
'''

def calculate_positive_negative_scores(text):
    # Load positive and negative words from files
    with open('positive-words.txt', 'r',encoding='latin') as file:
        positive_words = file.read().split()
    with open('negative-words.txt', 'r',encoding='latin') as file:
        negative_words = file.read().split()

    # Tokenize the text
    tokens = text.split()

    # Count positive and negative words
    positive_count = sum(1 for word in tokens if word in positive_words)
    negative_count = sum(1 for word in tokens if word in negative_words)

    # Calculate scores
    positive_score = positive_count / (positive_count + negative_count) if positive_count + negative_count > 0 else 0
    negative_score = negative_count / (positive_count + negative_count) if positive_count + negative_count > 0 else 0

    return positive_score, negative_score


In [ ]:
'''
performing the desired text analysis
TextBlob - polarity,subjectivity
complex words - words with more than 2 letters
fog_index - complexity of the sentence
syllable_per_word - avg no.of syllables per word
'''

def calculate_textual_metrics(text):
    # Preprocess text
    tokens = preprocess_text(text)

    # Sentiment analysis
    blob = TextBlob(text)
    polarity_score = blob.sentiment.polarity
    subjectivity_score = blob.sentiment.subjectivity

    # Average sentence length
    sentences = sent_tokenize(text)
    avg_sentence_length = sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences)

    # Percentage of complex words
    complex_words = [word for word in tokens if len(word) > 2]
    percentage_complex_words = len(complex_words) / len(tokens) * 100

    # Fog Index
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    # Average number of words per sentence
    avg_words_per_sentence = sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences)

    # Complex word count
    complex_word_count = len(complex_words)

    # Word count
    word_count = len(tokens)

    # Syllable per word
    dic = pyphen.Pyphen(lang='en_US')
    syllable_per_word = sum(len(dic.inserted(word).split('-')) for word in tokens) / word_count

    # Personal pronouns
    personal_pronouns = sum(1 for word in tokens if word in ['I', 'we', 'my', 'ours', 'us'])

    # Average word length
    avg_word_length = sum(len(word) for word in tokens) / word_count

    positive_score, negative_score = calculate_positive_negative_scores(text)

    return pd.Series({
        'Polarity Score': polarity_score,
        'Subjectivity Score': subjectivity_score,
        'Avg Sentence Length': avg_sentence_length,
        'Percentage of Complex Words': percentage_complex_words,
        'Fog Index': fog_index,
        'Avg Number of Words per Sentence': avg_words_per_sentence,
        'Complex Word Count': complex_word_count,
        'Word Count': word_count,
        'Syllable per Word': syllable_per_word,
        'Personal Pronouns': personal_pronouns,
        'Avg Word Length': avg_word_length,
        'Positive Score': positive_score,
        'Negative Score': negative_score
    })

# Apply the function to each row in the DataFrame
nlp_analysis = url_df[1].apply(calculate_textual_metrics)
nlp_analysis.head()


,Polarity Score,Subjectivity Score,Avg Sentence Length,Percentage of Complex Words,Fog Index,Avg Number of Words per Sentence,Complex Word Count,Word Count,Syllable per Word,Personal Pronouns,Avg Word Length,Positive Score,Negative Score
0,0.284952,0.554210,17.589744,99.814815,46.961823,17.589744,539.0,540.0,1.996296,0.0,6.394444,0.891304,0.108696
1,0.125024,0.431500,21.112500,99.871465,48.393586,21.112500,777.0,778.0,2.289203,0.0,7.177378,0.701299,0.298701
2,0.100987,0.431038,21.368421,98.728140,48.038624,21.368421,621.0,629.0,2.607313,0.0,7.934817,0.648148,0.351852
3,0.041761,0.408749,23.442308,99.503311,49.178248,23.442308,601.0,604.0,2.554636,0.0,7.849338,0.355556,0.644444
4,0.117054,0.531510,19.538462,99.728997,47.706984,19.538462,368.0,369.0,2.200542,0.0,7.146341,0.720000,0.280000


In [ ]:
# saving the output as .csv file
nlp_analysis.to_csv('nlp_analysis_output.csv', index=False)